In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
import matplotlib.dates as dts
import numpy as np
import pandas as pd
import itertools
import os
import ROOT
import datetime
from root_numpy import root2array, root2rec, tree2rec, array2root
from scipy.optimize import curve_fit
from scipy.misc import factorial
plt.rcParams.update({'font.size': 16})

In [ ]:
#df = pd.read_pickle('stopmu.pkl')
#df = pd.read_pickle('stopmu_data_mcc83.pkl') # data
data = pd.read_csv('stopmu_mcc83.csv',sep='\t')
print data.shape

In [ ]:
data.columns = ['rr','dqdx','pitch','px','py','pz']

In [ ]:
fig = plt.figure(figsize=(12,6))
BINS = ( np.linspace(0,150,100), np.linspace(150,500,100) )
plt.hist2d(data['rr'].values,data['dqdx'].values,bins=BINS)
plt.grid()
plt.xlabel('Residual Range [ cm ]',fontsize=20,fontweight='bold')
plt.ylabel('dQ/dx [ ADC / cm ]',fontsize=20,fontweight='bold')
plt.title('Tagged Stopping Muon Profile [MC Cosmics]',fontsize=20,fontweight='bold')
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))
datamip = data.query('rr > 100 and rr < 140')
BINS = np.linspace(150,350,100)
plt.hist(datamip['dqdx'].values,bins=BINS,histtype='step',lw=2)
plt.grid()
plt.show()

In [ ]:
# invert Recombination Modified Box Model to get dE/dx from dQ/dx

# argon density [g/cm^3]
rho = 1.396
# electric field [kV/cm]
efield = 0.273
# ionization energy [MeV/e]
Wion = 23.6*(10**(-6))

fModBoxA = 0.93
fModBoxB = 0.562

def ModBoxInverse(dqdx):
    dedx = (np.exp(fModBoxB * Wion * dqdx ) - fModBoxA) / fModBoxB
    return dedx    

In [ ]:
# we know the true electronics gain in simulation, apply and then convert to dEdx

# electronics gain [e/ADC]
elecgain = 200.

def dEdx(x):
    dqdx = x['dqdx'] * elecgain
    return ModBoxInverse(dqdx)

In [ ]:
data['dedx'] = data.apply(lambda x : dEdx(x), axis=1)

In [ ]:
fig = plt.figure(figsize=(12,6))
BINS = ( np.linspace(0,150,100), np.linspace(1,3,100) )
plt.hist2d(data['rr'].values,data['dedx'].values,bins=BINS)
plt.grid()
plt.xlabel('Residual Range [ cm ]',fontsize=20,fontweight='bold')
plt.ylabel('dQ/dx [ MeV / cm ]',fontsize=20,fontweight='bold')
plt.title('Tagged Stopping Muon Profile [MC Cosmics]',fontsize=20,fontweight='bold')
plt.show()

In [ ]:
from larlite import larlite as ll

In [ ]:
def GL(x_v,mpv,sG,sL,A):
    
    lv_v = []
    
    for x in x_v:
    
        fgl = ROOT.GaussLandau(sG,sL,mpv,A)
    
        lv_v.append( fgl.at(x) )
    
    return lv_v

GL([2],1,1,1,2)

In [ ]:
#rr_range_v = np.linspace(10,150,40)

BINS = np.linspace(1,6.0,100)
xvals = np.linspace(1,6,100)

mpv_v = []
mpv_e = []
rr_v = []
rr_ranges = np.linspace(20,200,40)

for n in xrange(len(rr_ranges)-1):
    
    rrmin = rr_ranges[n]
    rrmax = rr_ranges[n+1]
    
    dftmp = data.query('rr > %i and rr < %i and pitch < 0.4'%(rrmin,rrmax))
    
    dedx_v = dftmp['dedx'].values
    
    vals,bine = np.histogram(dedx_v,bins=BINS)
    binc = 0.5*(bine[1:]+bine[:-1])
    guess = [1.6,0.1,0.1,8000.]
    popt,popv = curve_fit(GL,binc,vals,p0=guess)#,sigma=np.sqrt(vals),absolute_sigma=True)
    print popt
    
    pope = np.sqrt(np.diag(popv))
    
    mpv_v.append(popt[0])
    mpv_e.append(pope[0])
    rr_v.append(0.5*(rrmin+rrmax))
    
    fig = plt.figure(figsize=(6,6))
    plt.xlabel('dE/dx [MeV/cm]',fontsize=20,fontweight='bold')
    plt.errorbar(binc,vals,yerr=np.sqrt(vals),fmt='bo',lw=2)
    plt.title('Residual Range [%i,%i]'%(rrmin,rrmax),fontsize=20,fontweight='bold')
    plt.plot(xvals,GL(xvals,*popt),'r--',lw=2,label='MPV = %.02f MeV/cm'%(popt[0]))
    plt.grid()
    plt.legend(loc=1)
    plt.show()


In [ ]:
fig = plt.figure(figsize=(6,6))
plt.errorbar(rr_v,mpv_v,yerr=mpv_e,fmt='bo',lw=2,markersize=8)
plt.grid()
plt.xlabel('RR [cm]')
plt.ylabel('Fitted MPV [MeV/cm]')
plt.show()

In [ ]:
fout = open('/home/david/Desktop/dedx_vs_rr.txt','w+')
for i,rr in enumerate(rr_v):
    mpv  = mpv_v[i]
    mpve = mpv_e[i]
    fout.write('%.04f %.04f %.04f \n'%(rr,mpv,mpve))
fout.close()

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.hist(data['pitch'],bins=np.linspace(0.29,1.0,50),histtype='step',lw=2)
plt.grid()
plt.xlabel('Pitch [cm]',fontsize=20,fontweight='bold')
plt.ylabel('Entries',fontsize=20,fontweight='bold')
plt.title('Pitch Distribution',fontsize=20,fontweight='bold')
plt.axvline(0.4,lw=4,color='k',linestyle='--')
plt.text(0.42,22000,r'$\leftarrow$ cut',fontsize=26,fontweight='bold')
plt.show()

print np.median(data['pitch'].values)
print np.average(data['pitch'].values)

In [ ]:
#rr_range_v = np.linspace(10,150,40)

BINS = np.linspace(1,6.0,100)
xvals = np.linspace(1,6,100)

pitch_ranges_v = [[0.3,0.33,0.36,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.9],\
                 [0.3,0.4,0.5,0.6,0.7,0.8,0.9]]

#pitch_ranges_v = [[0.3,0.35,0.4,0.5,0.6,0.7,0.8,0.9],\
#                [0.3,0.35,0.4,0.5,0.6,0.7,0.8,0.9],\
#                [0.4,0.5,0.6,0.7,0.9]]

px_ranges = [0,0.3,1.0]#0.2,0.4,0.8]

fout = open('/home/david/Desktop/dedx_vs_pitch.txt','w+')

for m in xrange(len(px_ranges)-1):
    
    pxmin = px_ranges[m]
    pxmax = px_ranges[m+1]
    
    dfpx = data.query('rr > 100 and rr < 150. and ((px > %f and px < %f) or (px > %f and px < %f))'%\
    (pxmin,pxmax,-pxmax,-pxmin))
    
    mpv_v = []
    mpv_e = []
    pitchh_v = []
    pitchl_v = []
    pitch_v = []

    for n in xrange(len(pitch_ranges_v[m])-1):
    
        pmin = pitch_ranges_v[m][n]
        pmax = pitch_ranges_v[m][n+1]
    
        pitchh_v.append(pmax)
        pitchl_v.append(pmin)
        pitch_v.append(0.5*(pmin+pmax))
    
        dftmp = dfpx.query('rr > 100 and rr < 150 and pitch > %f and pitch < %f'%(pmin,pmax))
    
        dedx_v = dftmp['dedx'].values
    
        vals,bine = np.histogram(dedx_v,bins=BINS)
        binc = 0.5*(bine[1:]+bine[:-1])
        guess = [1.6,0.1,0.1,10000.]
        popt,popv = curve_fit(GL,binc,vals,p0=guess)#,sigma=np.sqrt(vals),absolute_sigma=True)
        print popt
    
        pope = np.sqrt(np.diag(popv))
    
        mpv_v.append(popt[0])
        mpv_e.append(pope[0])
    
        fig = plt.figure(figsize=(10,6))
        plt.errorbar(binc,vals,yerr=np.sqrt(vals),fmt='bo',lw=2)
        plt.title('Pitch range [%.02f,%.02f]'%(pmin,pmax))
        plt.plot(xvals,GL(xvals,*popt),'r--',lw=2)
        plt.grid()
        plt.show()
        
        
        fout.write('%.04f %.04f %.04f %.04f %.04f %.04f %.04f \n'%\
                   (pxmin,pxmax,0.5*(pmin+pmax),pmin,pmax,popt[0],pope[0]))
        
    fig = plt.figure(figsize=(6,6))
    plt.errorbar(pitch_v,mpv_v,yerr=mpv_e,fmt='bo',lw=2,markersize=8)
    plt.grid()
    plt.xlabel('Pitch [cm]')
    plt.ylabel('Fitted MPV [MeV/cm]')
    plt.title('Px in range [%.02f,%.02f]'%(pxmin,pxmax))
    plt.show()

fout.close()
